In [1]:
pip install pyyaml h5py  # Required to save models in HDF5 format

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.18.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]



In [4]:
train_images[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [5]:
train_images.shape

(60000, 28, 28)

In [6]:
train_labels.shape

(1000,)

-1: Infers batch size (automatically calculated).
28 × 28: Reshapes data to 784 features (e.g., 28x28 images).

In [7]:
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0  # flatten
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [8]:
train_images.shape

(1000, 784)

In [9]:
train_images[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [10]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

d:\a27_YEARS_OLD\deep_learning\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

Save checkpoints during training
You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The tf.keras.callbacks.ModelCheckpoint callback allows you to continually save the model both during and at the end of training.

In [11]:
# Checkpoint callback usage

checkpoint_path = "training_1/.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [12]:
checkpoint_dir

'training_1'

In [13]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [14]:




# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.6027 - sparse_categorical_accuracy: 0.5073 
Epoch 1: saving model to training_1/.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - loss: 1.5883 - sparse_categorical_accuracy: 0.5125 - val_loss: 0.7609 - val_sparse_categorical_accuracy: 0.7550
Epoch 2/10
23/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5144 - sparse_categorical_accuracy: 0.8450
Epoch 2: saving model to training_1/.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4837 - sparse_categorical_accuracy: 0.8570 - val_loss: 0.5482 - val_sparse_categorical_accuracy: 0.8180
Epoch 3/10
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2804 - sparse_categorical_accuracy: 0.9202
Epoch 3: saving model to training_1/.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.2795 - sparse_categorical_accuracy: 0.9208 - val_loss: 0.5043 - val_sparse_categorical_accuracy: 0.8370
Epoch 4/10
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2178 - sparse_categoric

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [15]:
os.listdir(checkpoint_dir)

['.weights.h5']

As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights.

Now rebuild a fresh, untrained model and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [16]:
# Create a basic model instance
model2 = create_model()

# Evaluate the model
loss, acc = model2.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 9ms/step - loss: 2.3585 - sparse_categorical_accuracy: 0.0790
Untrained model, accuracy:  7.90%


Then load the weights from the checkpoint and re-evaluate:

In [17]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 3ms/step - loss: 0.4123 - sparse_categorical_accuracy: 0.8680
Restored model, accuracy: 86.80%


In [18]:
# Loads the weights
model2.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model2.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 3ms/step - loss: 0.4123 - sparse_categorical_accuracy: 0.8680
Restored model, accuracy: 86.80%


d:\a27_YEARS_OLD\deep_learning\venv\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Checkpoint callback options
The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [33]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [34]:
checkpoint_path

'training_2/cp-{epoch:04d}.weights.h5'

In [35]:
checkpoint_dir

'training_2'

In [36]:

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)


In [37]:
# Create a new model instance
model3 = create_model()

In [38]:

# Save the weights using the `checkpoint_path` format
model3.save_weights(checkpoint_path.format(epoch=0))

In [39]:

# Train the model with the new callback
model3.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.weights.h5

Epoch 10: saving model to training_2/cp-0010.weights.h5

Epoch 15: saving model to training_2/cp-0015.weights.h5

Epoch 20: saving model to training_2/cp-0020.weights.h5

Epoch 25: saving model to training_2/cp-0025.weights.h5

Epoch 30: saving model to training_2/cp-0030.weights.h5

Epoch 35: saving model to training_2/cp-0035.weights.h5

Epoch 40: saving model to training_2/cp-0040.weights.h5

Epoch 45: saving model to training_2/cp-0045.weights.h5

Epoch 50: saving model to training_2/cp-0050.weights.h5


In [40]:
os.listdir(checkpoint_dir)

['cp-0000.weights.h5',
 'cp-0005.weights.h5',
 'cp-0010.weights.h5',
 'cp-0015.weights.h5',
 'cp-0020.weights.h5',
 'cp-0025.weights.h5',
 'cp-0030.weights.h5',
 'cp-0035.weights.h5',
 'cp-0040.weights.h5',
 'cp-0045.weights.h5',
 'cp-0050.weights.h5']

In [45]:
os.listdir(checkpoint_dir)[-1]

'cp-0050.weights.h5'

In [51]:
latest = os.path.join('training_2',os.listdir(checkpoint_dir)[-1])
latest

'training_2\\cp-0050.weights.h5'

In [ ]:
# latest = tf.train.latest_checkpoint(checkpoint_path)
# latest

To test, reset the model, and load the latest checkpoint:

In [56]:
# Create a new model instance
model4 = create_model()

In [57]:
print(latest)

training_2\cp-0050.weights.h5


In [58]:
# Load the previously saved weights
model4.load_weights(latest)

d:\a27_YEARS_OLD\deep_learning\venv\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [59]:
# Re-evaluate the model
loss, acc = model4.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 6ms/step - loss: 0.4734 - sparse_categorical_accuracy: 0.8750
Restored model, accuracy: 87.50%


In [63]:
# Save the weights
model4.save_weights('checkpoints/.weights.h5')

# Create a new model instance
model5 = create_model()

# Restore the weights
model5.load_weights('checkpoints/.weights.h5')

# Evaluate the model
loss, acc = model5.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

d:\a27_YEARS_OLD\deep_learning\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 - 0s - 6ms/step - loss: 0.4734 - sparse_categorical_accuracy: 0.8750
Restored model, accuracy: 87.50%


Save the entire model
Call tf.keras.Model.save to save a model's architecture, weights, and training configuration in a single model.keras zip archive.

In [64]:
# Create and train a new model instance.
model6 = create_model()
model6.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model6.save('my_model.keras')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.6381 - sparse_categorical_accuracy: 0.4760
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4928 - sparse_categorical_accuracy: 0.8590
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2947 - sparse_categorical_accuracy: 0.9285
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2335 - sparse_categorical_accuracy: 0.9557
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1429 - sparse_categorical_accuracy: 0.9807


In [65]:
model7 = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
model7.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,221,152 (4.66 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 814,102 (3.11 MB)

Try running evaluate and predict with the loaded model:

In [66]:
# Evaluate the restored model
loss, acc = model7.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(model7.predict(test_images).shape)

32/32 - 0s - 5ms/step - loss: 0.4471 - sparse_categorical_accuracy: 0.8550
Restored model, accuracy: 85.50%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
(1000, 10)


In [69]:
model7.predict(test_images)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[-1.8644843 , -5.3490186 , -1.4585346 , ...,  8.047205  ,
        -1.6924257 ,  2.0203164 ],
       [-0.05265317, -1.2017248 ,  4.0894113 , ..., -8.760731  ,
         1.2358156 , -4.282245  ],
       [-3.7238638 ,  4.808173  ,  0.649599  , ..., -0.354739  ,
         0.2669912 , -1.6323596 ],
       ...,
       [11.296146  , -4.45019   , -3.001043  , ..., -5.817418  ,
        -0.17724665, -3.1806602 ],
       [-1.5575032 , -5.4145193 , -4.409949  , ..., -0.37405342,
         3.472923  ,  2.9945953 ],
       [-3.612101  , -4.0416613 , -1.209384  , ...,  3.134431  ,
        -1.0986335 ,  2.892395  ]], dtype=float32)

In [70]:
test_images[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [71]:
(train_images1, train_labels1), (test_images1, test_labels1) = tf.keras.datasets.mnist.load_data()

In [73]:
test_images1.shape

(10000, 28, 28)

In [75]:
test_images1[0].shape

(28, 28)

In [77]:
pred_img = test_images1[0].reshape(-1, 28 * 28) / 255.0

In [78]:
pred_img.shape

(1, 784)

In [80]:
out_img = model7.predict(pred_img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


In [81]:
import numpy as np

In [ ]:
np.argmax(out_img) # the digits is 7

np.int64(7)

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model.h5')

In [83]:
# my_model directory
# ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
# ls saved_model/my_model

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model.h5')

# Check its architecture
new_model.summary()

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)